In [ ]:
pip install meteostat -U

In [1]:
import pandas as pd
import numpy as np
import time 
from time import sleep
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from datetime import datetime, timedelta
from meteostat import Stations, Hourly, Point

In [2]:
#League / Season Summary URLs

seriea_urls = [
    "https://fbref.com/en/comps/11/2021-2022/schedule/2021-2022-Serie-A-Scores-and-Fixtures",
    "https://fbref.com/en/comps/11/2020-2021/schedule/2020-2021-Serie-A-Scores-and-Fixtures",
    "https://fbref.com/en/comps/11/2019-2020/schedule/2019-2020-Serie-A-Scores-and-Fixtures",
    "https://fbref.com/en/comps/11/2018-2019/schedule/2018-2019-Serie-A-Scores-and-Fixtures",
    "https://fbref.com/en/comps/11/2017-2018/schedule/2017-2018-Serie-A-Scores-and-Fixtures"
]

bundesliga_urls = [
    "https://fbref.com/en/comps/20/2021-2022/schedule/2021-2022-Bundesliga-Scores-and-Fixtures",
    "https://fbref.com/en/comps/20/2020-2021/schedule/2020-2021-Bundesliga-Scores-and-Fixtures",
    "https://fbref.com/en/comps/20/2019-2020/schedule/2019-2020-Bundesliga-Scores-and-Fixtures",
    "https://fbref.com/en/comps/20/2018-2019/schedule/2018-2019-Bundesliga-Scores-and-Fixtures",
    "https://fbref.com/en/comps/20/2017-2018/schedule/2017-2018-Bundesliga-Scores-and-Fixtures"
]

epl_urls = [
    "https://fbref.com/en/comps/9/2021-2022/schedule/2021-2022-Premier-League-Scores-and-Fixtures",
    "https://fbref.com/en/comps/9/2020-2021/schedule/2020-2021-Premier-League-Scores-and-Fixtures",
    "https://fbref.com/en/comps/9/2019-2020/schedule/2019-2020-Premier-League-Scores-and-Fixtures",
    "https://fbref.com/en/comps/9/2018-2019/schedule/2018-2019-Premier-League-Scores-and-Fixtures",
    "https://fbref.com/en/comps/9/2017-2018/schedule/2017-2018-Premier-League-Scores-and-Fixtures" 
]

laliga_urls = [
    "https://fbref.com/en/comps/12/2021-2022/schedule/2021-2022-La-Liga-Scores-and-Fixtures",
    "https://fbref.com/en/comps/12/2020-2021/schedule/2020-2021-La-Liga-Scores-and-Fixtures",
    "https://fbref.com/en/comps/12/2019-2020/schedule/2019-2020-La-Liga-Scores-and-Fixtures",
    "https://fbref.com/en/comps/12/2018-2019/schedule/2018-2019-La-Liga-Scores-and-Fixtures",
    "https://fbref.com/en/comps/12/2017-2018/schedule/2017-2018-La-Liga-Scores-and-Fixtures"
]

ligue1_urls = [
    "https://fbref.com/en/comps/13/2021-2022/schedule/2021-2022-Ligue-1-Scores-and-Fixtures",
    "https://fbref.com/en/comps/13/2020-2021/schedule/2020-2021-Ligue-1-Scores-and-Fixtures",
    "https://fbref.com/en/comps/13/2019-2020/schedule/2019-2020-Ligue-1-Scores-and-Fixtures",
    "https://fbref.com/en/comps/13/2018-2019/schedule/2018-2019-Ligue-1-Scores-and-Fixtures",
    "https://fbref.com/en/comps/13/2017-2018/schedule/2017-2018-Ligue-1-Scores-and-Fixtures"
]

league_urls = {    
    'Serie A': seriea_urls,
    'Bundesliga': bundesliga_urls,
    'EPL': epl_urls,
    'La Liga': laliga_urls,
    "Ligue 1": ligue1_urls
}

In [3]:
#For each League / Season summary page, extracts the Match Report URLs as df with column for leage  ***WORKS!!!!***
match_report_urls = pd.DataFrame()

for league_name, urls in league_urls.items(): #change: test_urls -> league_urls
    for url in urls: 
        sleep(1)
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        table = soup.find('table')

        links = []
        data = pd.DataFrame()
        for tr in table.findAll("tr"):
            trs = tr.findAll("td", {'data-stat':'match_report'})
            for each in trs:
                try:
                    link = each.find('a')['href']
                    links.append(link)
                except:
                    pass

        #Adds the https prefix to the links
        new_links = []
        prefix = 'https://fbref.com/'
        new_links = [prefix + x for x in links]
        
        data = pd.DataFrame(new_links, columns={'URL'})
        data['League'] = league_name
        
        match_report_urls = pd.concat([match_report_urls,data],axis=0)

match_report_urls

,URL,League
0,https://fbref.com//en/matches/5c162318/Interna...,Serie A
1,https://fbref.com//en/matches/81074285/Hellas-...,Serie A
2,https://fbref.com//en/matches/0465e44e/Torino-...,Serie A
3,https://fbref.com//en/matches/e057a19f/Empoli-...,Serie A
4,https://fbref.com//en/matches/015e7a96/Bologna...,Serie A
...,...,...
377,https://fbref.com//en/matches/e9cc3461/Lyon-Ni...,Ligue 1
378,https://fbref.com//en/matches/fbd790ad/Toulous...,Ligue 1
379,https://fbref.com//en/matches/fe96d78a/Troyes-...,Ligue 1
380,https://fbref.com//en/matches/f927719d/Ajaccio...,Ligue 1


In [4]:
match_report_urls = match_report_urls.reset_index(drop=True)

In [5]:
# For each Match Summary url, returns the home and away team stats ***WORKS!!!***

all_game_stats = pd.DataFrame()
admin_df = pd.DataFrame()

for i in range(len(match_report_urls)): #replace number with -> len(match_report_urls)
    url = match_report_urls.loc[i]['URL']   
    game_stats = pd.read_html(url)
    sleep(3)

    home_team_tables = game_stats[3:10]
    away_team_tables = game_stats[10:17]
    
    home_stats = pd.DataFrame()
    away_stats = pd.DataFrame()
    
    if len(home_team_tables) ==7 and len(away_team_tables)==7:
        
        for j in range(7): 
            temp_home = home_team_tables[j].iloc[-1]
            home_stats = pd.concat([home_stats, temp_home], axis=0)

        home_stats = home_stats.T
        home_column_maps = pd.read_csv('home_column_maps.csv')
        home_stats.columns = home_column_maps['new'] 
        home_stats = home_stats.rename_axis(None,axis=1)
        home_stats.drop(['todrop'], axis=1, inplace=True)
        home_stats['Home Team'] = game_stats[2].columns.get_level_values(0)[0]

        for k in range(7): 
            temp_away = away_team_tables[k].iloc[-1]
            away_stats = pd.concat([away_stats, temp_away], axis=0)

        away_stats = away_stats.T
        away_column_maps = pd.read_csv('away_column_maps.csv')
        away_stats.columns = away_column_maps['new'] 
        away_stats = away_stats.rename_axis(None,axis=1)
        away_stats.drop(['todrop'], axis=1, inplace=True)
        away_stats['Away Team'] = game_stats[2].columns.get_level_values(0)[1]   

        all_stats = pd.concat([home_stats, away_stats], axis=1)
        all_game_stats = pd.concat([all_game_stats, all_stats], axis=0)
        
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        score = soup.find("div", class_="scorebox")
        scorebox = soup.find("div", class_="scorebox_meta")

        weather_input = []
        date = scorebox.select_one('span[class="venuetime"]').get('data-venue-date')
        time = scorebox.select_one('span[class="venuetime"]').get('data-venue-time')
        venue = scorebox.find('small', string='Venue').find_next('small').text
        location = venue.rsplit(',')[-1]

        weather_input.extend([date,time,location])
        weather = pd.DataFrame(weather_input).T

        admin_df = pd.concat([admin_df, weather],axis=0)

HTTPError: HTTP Error 404: Not Found

In [6]:
copy_all_game_stats = all_game_stats
copy_admin_df = admin_df

In [13]:
copy_all_game_stats2 = all_game_stats2
copy_admin_df2 = admin_df2

In [10]:
all_game_stats.to_csv('SP Match Stats.csv')
admin_df.to_csv('SP Admin Stats.csv')

In [14]:
all_game_stats2.to_csv('SP Match Stats2.csv')
admin_df2.to_csv('SP Admin Stats2.csv')

In [12]:
# For each Match Summary url, returns the home and away team stats ***WORKS!!!***

all_game_stats2 = pd.DataFrame()
admin_df2 = pd.DataFrame()

for i in range(5621, len(match_report_urls)): #replace number with -> len(match_report_urls)
    url = match_report_urls.loc[i]['URL']   
    game_stats = pd.read_html(url)
    sleep(3)

    home_team_tables = game_stats[3:10]
    away_team_tables = game_stats[10:17]
    
    home_stats = pd.DataFrame()
    away_stats = pd.DataFrame()
    
    if len(home_team_tables) ==7 and len(away_team_tables)==7:
        
        for j in range(7): 
            temp_home = home_team_tables[j].iloc[-1]
            home_stats = pd.concat([home_stats, temp_home], axis=0)

        home_stats = home_stats.T
        home_column_maps = pd.read_csv('home_column_maps.csv')
        home_stats.columns = home_column_maps['new'] 
        home_stats = home_stats.rename_axis(None,axis=1)
        home_stats.drop(['todrop'], axis=1, inplace=True)
        home_stats['Home Team'] = game_stats[2].columns.get_level_values(0)[0]

        for k in range(7): 
            temp_away = away_team_tables[k].iloc[-1]
            away_stats = pd.concat([away_stats, temp_away], axis=0)

        away_stats = away_stats.T
        away_column_maps = pd.read_csv('away_column_maps.csv')
        away_stats.columns = away_column_maps['new'] 
        away_stats = away_stats.rename_axis(None,axis=1)
        away_stats.drop(['todrop'], axis=1, inplace=True)
        away_stats['Away Team'] = game_stats[2].columns.get_level_values(0)[1]   

        all_stats = pd.concat([home_stats, away_stats], axis=1)
        all_game_stats2 = pd.concat([all_game_stats2, all_stats], axis=0)
        
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        score = soup.find("div", class_="scorebox")
        scorebox = soup.find("div", class_="scorebox_meta")

        weather_input = []
        date = scorebox.select_one('span[class="venuetime"]').get('data-venue-date')
        time = scorebox.select_one('span[class="venuetime"]').get('data-venue-time')
        venue = scorebox.find('small', string='Venue').find_next('small').text
        location = venue.rsplit(',')[-1]

        weather_input.extend([date,time,location])
        weather = pd.DataFrame(weather_input).T

        admin_df2 = pd.concat([admin_df2, weather],axis=0)

HTTPError: HTTP Error 403: Forbidden

In [18]:
# For each Match Summary url, returns the home and away team stats ***WORKS!!!***

all_game_stats3 = pd.DataFrame()
admin_df3 = pd.DataFrame()

for i in range(9028, len(match_report_urls)): #replace number with -> len(match_report_urls)
    url = match_report_urls.loc[i]['URL']   
    game_stats = pd.read_html(url)
    sleep(3)

    home_team_tables = game_stats[3:10]
    away_team_tables = game_stats[10:17]
    
    home_stats = pd.DataFrame()
    away_stats = pd.DataFrame()
    
    if len(home_team_tables) ==7 and len(away_team_tables)==7:
        
        for j in range(7): 
            temp_home = home_team_tables[j].iloc[-1]
            home_stats = pd.concat([home_stats, temp_home], axis=0)

        home_stats = home_stats.T
        home_column_maps = pd.read_csv('home_column_maps.csv')
        home_stats.columns = home_column_maps['new'] 
        home_stats = home_stats.rename_axis(None,axis=1)
        home_stats.drop(['todrop'], axis=1, inplace=True)
        home_stats['Home Team'] = game_stats[2].columns.get_level_values(0)[0]

        for k in range(7): 
            temp_away = away_team_tables[k].iloc[-1]
            away_stats = pd.concat([away_stats, temp_away], axis=0)

        away_stats = away_stats.T
        away_column_maps = pd.read_csv('away_column_maps.csv')
        away_stats.columns = away_column_maps['new'] 
        away_stats = away_stats.rename_axis(None,axis=1)
        away_stats.drop(['todrop'], axis=1, inplace=True)
        away_stats['Away Team'] = game_stats[2].columns.get_level_values(0)[1]   

        all_stats = pd.concat([home_stats, away_stats], axis=1)
        all_game_stats3 = pd.concat([all_game_stats3, all_stats], axis=0)
        
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        score = soup.find("div", class_="scorebox")
        scorebox = soup.find("div", class_="scorebox_meta")

        weather_input = []
        date = scorebox.select_one('span[class="venuetime"]').get('data-venue-date')
        time = scorebox.select_one('span[class="venuetime"]').get('data-venue-time')
        venue = scorebox.find('small', string='Venue').find_next('small').text
        location = venue.rsplit(',')[-1]

        weather_input.extend([date,time,location])
        weather = pd.DataFrame(weather_input).T

        admin_df3 = pd.concat([admin_df3, weather],axis=0)

In [27]:
just_in_case = all_game_stats3
just_in_case_admin = admin_df3
len(just_in_case)

118

In [28]:
all_game_stats4 = pd.concat([all_game_stats, all_game_stats2, all_game_stats3], axis=0)
len(all_game_stats4)
all_game_stats4.to_csv('SP Match Stats_final.csv')

In [29]:
admin_df4 = pd.concat([admin_df, admin_df2, admin_df3], axis=0)
len(admin_df4)
admin_df4.to_csv('SP Admin Stats_final.csv')

In [34]:
admin_df4.columns=['Date','Time','Location']
admin_df4 = admin_df4.reset_index(drop=True)

In [36]:
geolocator = Nominatim(user_agent="geoapiExercises")

def geolocate(city):
    stadium_address = geolocator.geocode(city)
    stadium_lat = stadium_address.latitude
    stadium_long = stadium_address.longitude
    return stadium_lat, stadium_long

In [37]:
lat = []
long = []

for i in range(6676, len(all_game_stats4)): # change to len(match_report_url) later 
    stadium_lat, stadium_long = geolocate(admin_df4.iloc[i]['Location'])
    lat.append(stadium_lat)
    long.append(stadium_long)
    sleep(1)
    
admin_df5 = pd.concat([admin_df4,pd.DataFrame({'Latitude':lat, 'Longitude':long})], axis=1)

GeocoderServiceError: Non-successful status code 502

In [41]:
lat2 = []
long2 = []

for i in range(6676, len(all_game_stats4)): # change to len(match_report_url) later 
    stadium_lat, stadium_long = geolocate(admin_df4.iloc[i]['Location'])
    lat2.append(stadium_lat)
    long2.append(stadium_long)
    sleep(1)
    
#admin_df5 = pd.concat([admin_df4,pd.DataFrame({'Latitude':lat, 'Longitude':long})], axis=1)

In [55]:
copy_lat.extend(copy_lat2)
len(copy_lat)

9146

In [57]:
copy_long.extend(copy_long2)
len(copy_long)

9146

In [54]:
copy_lat = original_lat
copy_lat2 = lat2

In [56]:
copy_long = long
copy_long2 = long2

In [58]:
admin_df5 = pd.concat([admin_df4,pd.DataFrame({'Latitude':copy_lat, 'Longitude':copy_long})], axis=1)

In [60]:
admin_df5

,Date,Time,Location,Latitude,Longitude
0,2021-08-21,18:30,Milano,45.464194,9.189635
1,2021-08-21,18:30,Verona,45.438496,10.992412
2,2021-08-21,20:45,Torino,45.067755,7.682489
3,2021-08-21,20:45,Empoli,43.719488,10.945856
4,2021-08-22,18:30,Bologna,44.493820,11.342633
...,...,...,...,...,...
9141,2018-05-19,21:00,Nantes,47.218637,-1.554136
9142,2018-05-19,21:00,Longeville-lès-Metz,49.115245,6.136124
9143,2018-05-19,21:00,Décines-Charpieu,45.773606,4.963331
9144,2018-05-19,21:00,Toulouse,43.604462,1.444247


In [62]:
weather = pd.DataFrame()

for i in range(len(all_game_stats4)): #replace with len(match_report_url)
    search = Point(admin_df5.iloc[i]['Latitude'], admin_df5.iloc[i]['Longitude'])
    date = admin_df5.iloc[i]['Date']
    time = admin_df5.iloc[i]['Time']
    combined = date + " " + time
    date_time_string = datetime.strptime(combined, '%Y-%m-%d %H:%M')
    
    start_time = date_time_string
    end_time = date_time_string + timedelta(hours=2)
    data = Hourly(search, start_time, end_time)
    wx_data = data.fetch()
    sleep(1)
    
    weather = pd.concat([weather, pd.DataFrame(wx_data.max()).T], axis=0)

In [ ]:
raw_data = pd.concat([match_report_urls['League'],admin_df4.reset_index(drop=True), all_game_stats4.reset_index(drop=True), weather.reset_index(drop=True)], axis=1)
raw_data.to_csv('SP Raw Data.csv')

In [63]:
raw_data = pd.concat([admin_df4.reset_index(drop=True), all_game_stats4.reset_index(drop=True), weather.reset_index(drop=True)], axis=1)
raw_data.to_csv('SP Raw Data.csv')